In [ ]:
import pandas as pd
import csv
import json

In [ ]:
# Function to read the 'GMNS link geometry' column and 
# convert it to a new 'GEOJSON geometry format' column

def GMNSLinkToGEOJSON(csvLinkFile, jsonExportFile):
    
    df = pd.read_csv(csvLinkFile)
    geoList = df['geometry'].tolist()
    new_column_ls = []
    
    for line in geoList:
        line = line[12:-1]
        line = [line]
        line_points_ls = []
    #     print('line:', line) # line is 1 element list 
        points = [ele.split(',') for ele in line]
        points = points[0]
        points = [ele.split(',') for ele in points]
        for point in points:
            point = [ele.split(' ')[-2:] for ele in point][0]
            point = [float(x) for x in point]
            line_points_ls.append(point)
#         print(line_points_ls)
        new_column_ls.append(line_points_ls)
    
    print('*',len(new_column_ls), 'links are identified.')
    df['coordinates'] = new_column_ls
    exportCSVFile = csvLinkFile[:-4] + '_modified.csv'
    df.to_csv(exportCSVFile, index = False)
    print('* Modified link file is saved to', exportCSVFile)
    
    # create a dictionary
    geojson = {
        'type': 'FeatureCollection',
        "name": "imported",
        'features': []
    }

    # Open a csv reader
    with open(exportCSVFile, encoding='utf-8') as csvf:
        csvRead = csv.DictReader(csvf)
        for rows in csvRead:
            coordinates = rows['coordinates']
            type_ = "Feature"
            geometry = {'type': 'LineString', 'coordinates': json.loads(coordinates)}
            properties = rows
            line_feature = {"type": "Feature",  "geometry": geometry, "properties": properties}
            geojson['features'].append(line_feature)

    with open(jsonExportFile, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(geojson, indent=2))
        
    print('* JSON file is saved to', jsonExportFile)
    print('* Done')

In [ ]:
csvLinkFile = r'C:\Users\lzhai\Working\QGIS_files\no residential\link.csv'
jsonExportFile = r'C:\Users\lzhai\Working\QGIS_files\no residential\link.json'
GMNSLinkToGEOJSON(csvLinkFile, jsonExportFile)